In [32]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.utils import (
    saved_model_export_utils)
from tensorflow.contrib.training.python.training import hparam
import tensorflow.contrib.layers as lays

In [33]:
CSV_COLUMN_DEFAULTS = [['']] + [[0.0] for i in range(1476)]

In [34]:
def autoencoder(inputs):
    # encoder
    # 28 x 28 x 1   ->  14 x 14 x 32
    # 14 x 14 x 32  ->  7 x 7 x 16
    # 7 x 7 x 16    ->  7 x 7 x 8
    net = lays.conv2d(inputs, 32, [5, 5], stride=1, padding='SAME')
    #print(net.get_shape())
    net = lays.conv2d(net, 16, [5, 5], stride=1, padding='SAME')
    #print(net.get_shape())
    net = lays.conv2d(net, 8, [5, 5], stride=1, padding='SAME')
    #print(net.get_shape())
    # decoder
    # 7 x 7 x 8    ->  7 x 7 x 16
    # 7 x 7 x 16   ->  14 x 14 x 32
    # 14 x 14 x 32  ->  28 x 28 x 1
    net = lays.conv2d_transpose(net, 16, [5, 5], stride=1, padding='SAME')
    #print(net.get_shape())
    net = lays.conv2d_transpose(net, 32, [5, 5], stride=1, padding='SAME')
    #print(net.get_shape())
    net = lays.conv2d_transpose(net, 1, [5, 5], stride=1, padding='SAME', activation_fn=tf.nn.tanh)
    #print(net.get_shape())
    return net

In [69]:
def model_fn(features, mode, params):
    ae_inputs = tf.reshape(features['x'], [-1, 6, 246, 1])
    ae_outputs = autoencoder(ae_inputs)  # create the Autoencoder network
    
    # calculate the loss and optimize the network
    loss = tf.reduce_mean(tf.square(ae_outputs - ae_inputs))  # claculate the mean square error loss
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        in_out_dist = tf.norm(ae_inputs - ae_outputs, axis=1)
        
        predictions = {
            'score': tf.reduce_sum(in_out_dist, 1),
            'hint_index': tf.argmax(in_out_dist, 1),
            'pid': features['pid']
            #'x': features['x'],
        }
        export_outputs = {
            'predict': tf.estimator.export.PredictOutput(predictions)
        }
        
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        'mse':tf.metrics.mean_squared_error(ae_inputs, ae_outputs)
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [70]:
def csv_serving_input_fn():
    csv_row = tf.placeholder(
        shape=[None],
        dtype=tf.string
    )
    
    features = parse_csv(csv_row)
    
    return tf.estimator.export.ServingInputReceiver(features, {'x': csv_row})

In [71]:
def json_serving_input_fn():
    inputs = {}
    inputs['x'] = tf.placeholder(shape=[None, 1476], dtype=tf.float32)   
    inputs['pid'] = tf.placeholder(shape=[None], dtype=tf.string)
      
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

In [72]:
SERVING_FUNCTIONS = {
    'JSON': json_serving_input_fn,
    'CSV': csv_serving_input_fn
}

In [73]:
def parse_csv(rows_string_tensor):

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=CSV_COLUMN_DEFAULTS)
    pid = columns.pop(0)
    columns = tf.concat(columns, axis=0)
        
    return {'x':columns, 'pid':pid}

In [74]:
def input_fn(filenames,
                        num_epochs=None,
                        shuffle=True,
                        skip_header_lines=0,
                        batch_size=200):
    filename_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    if shuffle:
      # Process the files in a random order.
      filename_dataset = filename_dataset.shuffle(len(filenames))
      
    # For each filename, parse it into one element per line, and skip the header
    # if necessary.
    dataset = filename_dataset.flat_map(
        lambda filename: tf.data.TextLineDataset(filename).skip(skip_header_lines))
    
    dataset = dataset.map(parse_csv)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=batch_size * 10)
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    features = iterator.get_next()
    
    return features

In [75]:
def build_estimator(run_config, hparams):
    estimator = tf.estimator.Estimator(model_fn=model_fn, 
                                  params=hparams, 
                                  config=run_config)

    return estimator

In [76]:
def run_experiment(hparams):
    """Run the training and evaluate using the high level API"""
  
    train_input = lambda: input_fn(
        hparams.train_files,
        num_epochs=hparams.num_epochs,
        batch_size=hparams.train_batch_size
    )
  
    # Don't shuffle evaluation data
    eval_input = lambda: input_fn(
        hparams.eval_files,
        batch_size=hparams.eval_batch_size,
        shuffle=False
    )
  
    train_spec = tf.estimator.TrainSpec(train_input,
                                        max_steps=hparams.train_steps
                                        )
  
    exporter = tf.estimator.FinalExporter('census',
            SERVING_FUNCTIONS[hparams.export_format])
    eval_spec = tf.estimator.EvalSpec(eval_input,
                                      steps=hparams.eval_steps,
                                      exporters=[exporter],
                                      name='census-eval'
                                      )
  
    run_config = tf.estimator.RunConfig()
    run_config = run_config.replace(model_dir=hparams.job_dir)
    
    estimator = build_estimator(
        run_config=run_config, hparams=hparams
    )
  
    tf.estimator.train_and_evaluate(estimator,
                                    train_spec,
                                    eval_spec)

In [77]:
hparams = tf.contrib.training.HParams(
    job_dir='output',
    train_files=['data/mff_20180506_seg1_train_pid.csv'],
    eval_files=['data/mff_20180506_seg1_eval_pid.csv'],
    train_steps=1,
    eval_steps=1,
    export_format='JSON',
    #export_format='CSV',
    num_epochs=5,
    train_batch_size=512,
    eval_batch_size=32
)

In [78]:
!rm -rf output
run_experiment(hparams)

INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_task_id': 0, '_service': None, '_num_ps_replicas': 0, '_master': '', '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_model_dir': 'output', '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7200f7a710>, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
[<tf.Tensor 'Conv/Relu:0' shape=(?, 6, 246, 32) dtype=float32>]
[<tf.Tensor 'Conv/Relu:0' shape=(?, 6, 246, 32) dtype=float32>, <tf.Tensor 'Conv_1/Relu:0' shape=(?, 6, 246, 32) dtype=float32>]
[<tf.Tensor 'Conv/Relu:0' shape=(?, 6, 246, 32) dtype=float32>,

In [ ]:
#!gcloud ml-engine local predict --model-dir=output/export/census/1533192404 --text-instances=data/mff_pred.csv
!gcloud ml-engine local predict --model-dir=output/export/census/1533528363 --json-instances=data/mff_pred.json